In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from IPython.display import HTML

# Read the file
file = pd.read_json('../raw_data.json')
series = pd.Series(file['data'])

# convert json to dataframe
df = pd.json_normalize(series)

print("Number of posts: ", len(df))

Number of posts:  3768


In [2]:
salaries_df = pd.DataFrame({'name': [], 'salary': [], 'currency': [], 'date': [], 'period': [], 'url': []})



# keywords
salary_keywords = ['Zarobki', 'zarobki', 'ZAROBKI']
currency_keywords = ['ZŁ', 'Zł', 'zł', 'USD','Usd', 'usd', '$','EURO', 'Euro', 'euro', 'EUR', 'Eur', 'eur', '€']
thousands = ['k']

def find_salaries(post):
    # find all salary infos in specific post
    # search through the every line in post
    # add new row to dataframe only if salaries is found
    # default false
    
    soup = BeautifulSoup(post['post_content'], 'html.parser')
    
    # remove blockquotes and avoid repetition
    for blockquote in soup.find_all('blockquote'):
        blockquote.decompose()
        
    # get clean text without html tags
    clean_text = soup.get_text()
    
    result = pd.DataFrame(columns = salaries_df.columns)
    
    found = False
    
    new_row = {'name': post['author'], 'salary': 0, 'url': post['url']}
    
    for line in clean_text.split('\n'):
        for keyword in salary_keywords:
            # check if salary is present
            if keyword in line:
                
                # regexp rules
                full_salary_match = re.search(r'\d+', line)
                short_salary_match = re.search(r'(\d+[.,]?\d*)\s*k', line)
                
                
                # find salary and convert it to int
                if(full_salary_match):
                    new_row['salary'] = int(full_salary_match.group(0))
                    found = True
                    
                elif (short_salary_match):
                    value = short_salary_match.group(0)
                
                    # convert , to . for Python float format
                    if "," in value:
                        value = value.replace(',', '.')
                        
                    # convert to full value, e.g. 6,5k -> 6500
                    if "k" in value:
                        value = value.replace('k', "")
                        value = float(value) * 1000
                        value = int(value)
                    
                    
                    new_row['salary'] = value
                    found = True
                    
                # search for currency of salary
                for currency in currency_keywords:
                    if currency in line:
                        new_row['currency'] = currency
                        found = True
                
                if(found):
                    result.loc[len(salaries_df)] = new_row
                    
            return result
    

# iterate over all posts
# analyze every post
for index, post in df.iterrows():
    
    post_salaries = find_salaries(post)
    
    salaries_df = pd.concat([salaries_df, post_salaries], ignore_index = True)
    
print('Processed posts: ', len(salaries_df))
HTML(salaries_df.to_html(render_links=True, escape=False))

Processed posts:  2


,name,salary,currency,date,period,url
0,Wybitny Szczur,7000,zł,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1154077#id1154077
1,Mały Mleczarz,10,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1154082#id1154082
